<a href="https://colab.research.google.com/github/aryan802/food_recognition_and_nutritional_analysis/blob/main/deep_learning_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
import zipfile
import os

# Mount Drive
drive.mount('/content/drive')

# Paths
zip_path = "/content/drive/MyDrive/archive (5).zip"  # change if in a different folder
extract_path = "/content/fast_food_data"

# Create folder and unzip
os.makedirs(extract_path, exist_ok=True)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Dataset extracted!")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dataset extracted!


In [4]:
train_dir = "/content/fast_food_data/Fast Food Classification V2/Train"
val_dir   = "/content/fast_food_data/Fast Food Classification V2/Valid"
test_dir  = "/content/fast_food_data/Fast Food Classification V2/Test"


In [9]:
from tensorflow.keras.applications.efficientnet import preprocess_input

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)


In [10]:
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2
)

val_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_gen = val_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

num_classes = len(train_gen.class_indices)
print("Number of classes:", num_classes)


Found 15000 images belonging to 10 classes.
Found 3500 images belonging to 10 classes.
Found 1500 images belonging to 10 classes.
Number of classes: 10


In [11]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224,224,3))

# Freeze base model initially
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_2         │ (None, 224, 224,  │          0 │ input_layer_1[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization_1     │ (None, 224, 224,  │          7 │ rescaling_2[0][0] │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_3         │ (None, 224, 224,  │          0 │ normalization_1[… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 225, 225,  │          0 │ rescaling_3[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 112, 112,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 112, 112,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 112, 112,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 112, 112,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 112, 112,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 112, 112,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 112, 112,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 112, 112,  │        512 │ block1a_se_excit

 Total params: 4,062,381 (15.50 MB)

 Trainable params: 12,810 (50.04 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [12]:
checkpoint = ModelCheckpoint("best_model.h5", monitor='val_accuracy', save_best_only=True, verbose=1)
early_stop = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.5, patience=3, verbose=1)

callbacks = [checkpoint, early_stop, reduce_lr]

In [13]:
initial_epochs = 10

history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=initial_epochs,
    callbacks=callbacks
)

Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 397ms/step - accuracy: 0.5966 - loss: 1.2763
Epoch 1: val_accuracy improved from -inf to 0.79800, saving model to best_model.h5


469/469 ━━━━━━━━━━━━━━━━━━━━ 223s 434ms/step - accuracy: 0.5969 - loss: 1.2756 - val_accuracy: 0.7980 - val_loss: 0.6690 - learning_rate: 0.0010
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 376ms/step - accuracy: 0.8001 - loss: 0.6394
Epoch 2: val_accuracy improved from 0.79800 to 0.81829, saving model to best_model.h5


469/469 ━━━━━━━━━━━━━━━━━━━━ 181s 386ms/step - accuracy: 0.8001 - loss: 0.6393 - val_accuracy: 0.8183 - val_loss: 0.6113 - learning_rate: 0.0010
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step - accuracy: 0.8190 - loss: 0.5778
Epoch 3: val_accuracy improved from 0.81829 to 0.82629, saving model to best_model.h5


469/469 ━━━━━━━━━━━━━━━━━━━━ 183s 389ms/step - accuracy: 0.8190 - loss: 0.5778 - val_accuracy: 0.8263 - val_loss: 0.5804 - learning_rate: 0.0010
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 380ms/step - accuracy: 0.8279 - loss: 0.5455
Epoch 4: val_accuracy did not improve from 0.82629
469/469 ━━━━━━━━━━━━━━━━━━━━ 183s 390ms/step - accuracy: 0.8279 - loss: 0.5455 - val_accuracy: 0.8243 - val_loss: 0.5952 - learning_rate: 0.0010
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step - accuracy: 0.8336 - loss: 0.5381
Epoch 5: val_accuracy improved from 0.82629 to 0.82800, saving model to best_model.h5


469/469 ━━━━━━━━━━━━━━━━━━━━ 182s 389ms/step - accuracy: 0.8336 - loss: 0.5380 - val_accuracy: 0.8280 - val_loss: 0.5793 - learning_rate: 0.0010
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 376ms/step - accuracy: 0.8370 - loss: 0.5086
Epoch 6: val_accuracy improved from 0.82800 to 0.83886, saving model to best_model.h5


469/469 ━━━━━━━━━━━━━━━━━━━━ 182s 388ms/step - accuracy: 0.8370 - loss: 0.5086 - val_accuracy: 0.8389 - val_loss: 0.5545 - learning_rate: 0.0010
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 375ms/step - accuracy: 0.8357 - loss: 0.5182
Epoch 7: val_accuracy did not improve from 0.83886
469/469 ━━━━━━━━━━━━━━━━━━━━ 181s 387ms/step - accuracy: 0.8357 - loss: 0.5182 - val_accuracy: 0.8357 - val_loss: 0.5718 - learning_rate: 0.0010
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 374ms/step - accuracy: 0.8404 - loss: 0.4913
Epoch 8: val_accuracy did not improve from 0.83886
469/469 ━━━━━━━━━━━━━━━━━━━━ 180s 383ms/step - accuracy: 0.8404 - loss: 0.4913 - val_accuracy: 0.8323 - val_loss: 0.5781 - learning_rate: 0.0010
Epoch 9/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 378ms/step - accuracy: 0.8435 - loss: 0.4987
Epoch 9: val_accuracy improved from 0.83886 to 0.84000, saving model to best_model.h5


469/469 ━━━━━━━━━━━━━━━━━━━━ 182s 388ms/step - accuracy: 0.8436 - loss: 0.4986 - val_accuracy: 0.8400 - val_loss: 0.5488 - learning_rate: 0.0010
Epoch 10/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 0s 379ms/step - accuracy: 0.8501 - loss: 0.4744
Epoch 10: val_accuracy did not improve from 0.84000
469/469 ━━━━━━━━━━━━━━━━━━━━ 182s 389ms/step - accuracy: 0.8501 - loss: 0.4744 - val_accuracy: 0.8377 - val_loss: 0.5594 - learning_rate: 0.0010
Restoring model weights from the end of the best epoch: 9.


In [16]:
test_loss, test_acc = model.evaluate(test_gen)
print("Test Accuracy:", test_acc)

47/47 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - accuracy: 0.9004 - loss: 0.3483
Test Accuracy: 0.8546666502952576


In [18]:
# # Mount Google Drive
# from google.colab import drive
# drive.mount('/content/drive')

# Save the trained model to Drive
model_save_path = "/content/drive/MyDrive/fast_food_classifier.h5"
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

# -----------------------------------------------
# # To load the model later for inference, use:
# from tensorflow.keras.models import load_model
# model = load_model("/content/drive/MyDrive/fast_food_classifier.h5")
# -----------------------------------------------



Model saved to /content/drive/MyDrive/fast_food_classifier.h5


In [22]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
train_dir = "/content/drive/MyDrive/Fast Food Classification V2/Train"
valid_dir = "/content/drive/MyDrive/Fast Food Classification V2/Valid"
test_dir  = "/content/drive/MyDrive/Fast Food Classification V2/Test"
model_path = "/content/drive/MyDrive/fast_food_classifier.h5"

In [28]:
zip_path = "/content/drive/MyDrive/archive (5).zip"  # Update with your Drive path
extract_path = "fast_food_data"

os.makedirs(extract_path, exist_ok=True)

with zipfile.ZipFile(zip_path, "r") as zip_ref:
    zip_ref.extractall(extract_path)

# Check structure
!ls fast_food_data/Fast\ Food\ Classification\ V2/Train


'Baked Potato'	'Crispy Chicken'   Fries      Pizza	 Taco
 Burger		 Donut		  'Hot Dog'   Sandwich	 Taquito


In [29]:
# from tensorflow.keras.models import load_model
# model = load_model("/content/drive/MyDrive/fast_food_model.h5")

In [31]:
# ---------------------------
# DASHBOARD SETUP
# ---------------------------

import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from ipywidgets import widgets, VBox, HBox, Layout
from IPython.display import display, clear_output

# ---------------------------
# 1. Load trained model
# ---------------------------
model_path = "/content/drive/MyDrive/fast_food_model.h5"  # Update path if needed
model = load_model(model_path)

# ---------------------------
# 2. Define classes
# ---------------------------
class_names = ['burger', 'fries', 'pizza', 'taquito', 'baked_potato']  # Example 5 labels

# ---------------------------
# 3. Hardcoded nutritional facts
# ---------------------------
nutrition_db = {
    "burger": {
        "Calories": 295,
        "Protein": "17g",
        "Fat": "12g",
        "Carbs": "33g"
    },
    "fries": {
        "Calories": 312,
        "Protein": "4g",
        "Fat": "15g",
        "Carbs": "41g"
    },
    "pizza": {
        "Calories": 285,
        "Protein": "12g",
        "Fat": "10g",
        "Carbs": "36g"
    },
    "taquito": {
        "Calories": 180,
        "Protein": "7g",
        "Fat": "8g",
        "Carbs": "20g"
    },
    "baked_potato": {
        "Calories": 161,
        "Protein": "4g",
        "Fat": "0.2g",
        "Carbs": "37g"
    }
}

# ---------------------------
# 4. Prediction function
# ---------------------------
def predict_food(img_path):
    img = Image.open(img_path).convert('RGB')
    img = img.resize((224,224))
    img_array = np.array(img)/255.0
    img_array = np.expand_dims(img_array, axis=0)

    pred = model.predict(img_array)
    class_idx = np.argmax(pred)
    return class_names[class_idx], np.max(pred)

# ---------------------------
# 5. Interactive dashboard
# ---------------------------
upload = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='Upload Image'
)

output = widgets.Output()

def on_upload_change(change):
    with output:
        clear_output()
        for name, file_info in upload.value.items():
            # Save uploaded image temporarily
            img_path = f"/tmp/{name}"
            with open(img_path, "wb") as f:
                f.write(file_info['content'])

            # Predict
            pred_class, confidence = predict_food(img_path)

            # Display image and prediction
            img = Image.open(img_path)
            plt.imshow(img)
            plt.axis('off')
            plt.title(f"Prediction: {pred_class}\nConfidence: {confidence:.2f}")
            plt.show()

            # Display nutritional facts
            facts = nutrition_db.get(pred_class, {})
            print("Nutritional Facts:")
            for k, v in facts.items():
                print(f"{k}: {v}")

upload.observe(on_upload_change, names='_counter')
display(VBox([upload, output]))




FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/drive/MyDrive/fast_food_model.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)